In [2]:
import pandas as pd
# 1. Load the real data
df = pd.read_csv(r'D:\Projects\Fake Reveiw Detector\data\raw_reviews.csv')

### Text Preprocessing

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# 2. Setup cleaning tools
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...


In [8]:
def clean_review(text):
    # Lowercase and remove non-letters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    # Tokenize and remove stopwords + lemmatize
    words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return " ".join(words)

df['cleaned_text'] = df['text'].apply(clean_review)

print("Dataset Loaded Successfully!")
print("Cleaned Example:", df['cleaned_text'].iloc[0][:100], "...")

Dataset Loaded Successfully!
Cleaned Example: stayed one night getaway family thursday triple aaa rate steal th floor room complete plasma tv bose ...


In [10]:
df['cleaned_text'][0]

'stayed one night getaway family thursday triple aaa rate steal th floor room complete plasma tv bose stereo voss evian water gorgeous bathroomno tub fine u concierge helpful cannot beat location flaw breakfast pricey service slowhours four kid four adult friday morning even though two table restaurant food good worth wait would return heartbeat gem chicago'

### Preparing the Labels

In [11]:
# Convert 'deceptive' column: truthful -> 0, deceptive -> 1
df['label'] = df['deceptive'].map({'truthful': 0, 'deceptive': 1})

In [12]:
print("Data distribution:")
print(df['label'].value_counts())

Data distribution:
label
0    800
1    800
Name: count, dtype: int64


In [13]:
# Now we separate our features (X) and our target (y)
X = df['cleaned_text']
y = df['label']

### Text Vectorization

In [14]:
# Here we use TF-IDF (Term Frequency-Inverse Document Frequency).
# This doesn't just count words, it gives higher "weight" to unique words that help distinguish a fake review from a real one,
# while down-weighting common words.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [16]:
# 1. Initialize the Vectorizer
# We limit to 3000 features to keep the model efficient and prevent overfitting
tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1,2)) 

In [17]:
# 2. Transform the text into numbers (The Matrix)
X_tfidf = tfidf.fit_transform(df['cleaned_text']).toarray()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [19]:
print("Vectorization Complete!")
print(f"Feature Matrix Shape: {X_tfidf.shape}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

Vectorization Complete!
Feature Matrix Shape: (1600, 3000)
Training samples: 1280
Testing samples: 320


In [21]:
"""
Note: Here we use ngram_range=(1,2) By setting this, the model looks at single words ("bad") and pairs of words ("very bad").
This is crucial for this project because "Fake" reviews often use specific word combinations that "Real" reviews don't.
"""

'\nNote: Here we use ngram_range=(1,2) By setting this, the model looks at single words ("bad") and pairs of words ("very bad").\nThis is crucial for this project because "Fake" reviews often use specific word combinations that "Real" reviews don\'t.\n'

### Model Training

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [23]:
clf = MultinomialNB(alpha=1.0)

In [24]:
clf.fit(X_train, y_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [26]:
y_pred = clf.predict(X_test)

In [28]:
print("Training Complete!\n")
print("--- Model Evaluation ---")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Truthful (0)', 'Deceptive (1)']))

Training Complete!

--- Model Evaluation ---
Overall Accuracy: 89.69%

Classification Report:
               precision    recall  f1-score   support

 Truthful (0)       0.94      0.86      0.90       168
Deceptive (1)       0.86      0.94      0.90       152

     accuracy                           0.90       320
    macro avg       0.90      0.90      0.90       320
 weighted avg       0.90      0.90      0.90       320



### Save Model

In [30]:
import joblib

joblib.dump(clf, r'D:\Projects\Fake Reveiw Detector\models\fake_review_model.pkl')

joblib.dump(tfidf, r'D:\Projects\Fake Reveiw Detector\models\tfidf_vectorizer.pkl')

print("Model and Vectorizer saved successfully!")

Model and Vectorizer saved successfully!


### Hybrid Model

In [31]:
import numpy as np
from scipy.sparse import hstack

def extract_stylometry(text):
    words = text.split()
    if len(words) == 0: return 0, 0, 0
    
    # 1. Lexical Diversity (Unique words / Total words)
    diversity = len(set(words)) / len(words)
    
    # 2. Punctuation Density (Count of ! and ?)
    punct_count = (text.count('!') + text.count('?')) / len(words)
    
    # 3. Pronoun Ratio (Count of 'I', 'me', 'my', 'we', 'our')
    pronouns = ['i', 'me', 'my', 'we', 'our']
    pronoun_count = sum(1 for w in words if w.lower() in pronouns) / len(words)
    
    return [diversity, punct_count, pronoun_count]

# Apply to the original 'text' column and convert to a numpy array
style_features = np.array(df['text'].apply(extract_stylometry).tolist())

# Combine TF-IDF (3000 features) + Style (3 features)
X_combined = hstack([tfidf.transform(df['cleaned_text']), style_features])

print("Old Shape:", X_tfidf.shape) # Should be (1600, 3000)
print("New Combined Shape:", X_combined.shape) # Should be (1600, 3003)

Old Shape: (1600, 3000)
New Combined Shape: (1600, 3003)


In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [34]:
hybrid_clf = RandomForestClassifier(n_estimators=100, random_state=42)
hybrid_clf.fit(X_train_h, y_train_h)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [35]:
hybrid_acc = hybrid_clf.score(X_test_h, y_test_h)
print(f"Hybrid Model Accuracy: {hybrid_acc:.2%}")

Hybrid Model Accuracy: 84.06%


In [37]:
import joblib

# Overwrite the model file
joblib.dump(hybrid_clf, r'D:\Projects\Fake Reveiw Detector\models\fake_review_model.pkl')

# Save the vectorizer again
joblib.dump(tfidf, r'D:\Projects\Fake Reveiw Detector\models\tfidf_vectorizer.pkl')

print("Success! Smart Hybrid Model is now saved")

Success! Smart Hybrid Model is now saved
